In [ ]:
import numpy as np
import pandas as pd
import math as m

In [ ]:
# formula for using a Brownian bridge
def b_bridge(t1, t2, t , x1, x2, M):
    z = np.sqrt((t2 - t) * (t - t1) / (t2 - t1))* np.random.normal(0, 1, (2, M))
    return ((t2 - t) * x1 + (t - t1) * x2) / (t2 - t1) + z

In [ ]:
def TimeStepAdaption(Y_n, M, A, theta):
    global h, sig_t, B
    # half h
    if h[0] > h[1] or h[0]==0: 
        h = np.insert(h, 1, h[1]/2) # if h[0]=0, it means there still finding the first h
    else:
        h = np.insert(h, 1, h[0]/2) 
    # use Brownian bridge to get the Brownian increment for the new h  
    if h[0] > h[2] or h[0]==0:
        B = np.insert(B, 1, np.array([b_bridge(t1=0, t2=h[2], t=h[1], x1=B[0,:,:], x2=B[1,:,:], M=M)]), axis=0)
    else:
        B = np.insert(B, 1, np.array([b_bridge(t1=0, t2=h[0], t=h[1], x1=B[0,:,:], x2=B[1,:,:], M=M)]), axis=0)
        
    B2 = A[0]*B[1, 0,:] + A[1]*B[1, 1,:] # Brownian motion for B2 in the SABR model
    sig_t = np.insert(sig_t, 1, sig_t[0,:]*np.exp((-alpha**2 / 2)*h[1] + B2), axis=0) # sigma(t)
    B1 = sig_t[1,:]*B[1, 0,:] # Brownian motion for B2 in the SABR model
    
    # Check if square root is positive
    if np.all((((Y_n + B1)**2)/4) >= (((sig_t[1,:]**2) * h[1] * theta) / (2 * (1 - theta))))==True:
        B = np.delete(B, 0, axis=0)
        h = np.delete(h, 0, axis=0)
        sig_t = np.delete(sig_t, 0, axis=0)
        Y_n = ((Y_n + B1)/2) + np.sqrt((((Y_n + B1)**2)/4) - ((sig_t[0,:]**2) * h[0] * theta) / (2 * (1 - theta)))
        
    else:
        sig_t = np.delete(sig_t, 1, axis=0)
        Y_n, sig_t, B, h = TimeStepAdaption(Y_n=Y_n, M=M, A=A, theta=theta)


    B2 = A[0]*B[1, 1,:] + A[1]*B[1, 0,:] # Brownian motion for B2 in the SABR model
    sig_t = np.insert(sig_t, 1, sig_t[0,:]*np.exp((-alpha**2 / 2)*h[0] + B2), axis=0) # sigma(t)
    B1 = sig_t[1,:]*B[1, 0,:] # Brownian motion for B2 in the SABR model
    
    # Check if square root is positive
    if np.all((((Y_n + B1)**2)/4) >= (((sig_t[1,:]**2) * h[0] * theta) / (2 * (1 - theta))))==False:
        sig_t = np.delete(sig_t, 1, axis=0)
        Y_n, sig_t, B, h = TimeStepAdaption(Y_n=Y_n, M=M, A=A, theta=theta)
        return Y_n, sig_t, B, h
                          
    else:
        Y_n = ((Y_n + B1)/2) + np.sqrt((((Y_n + B1)**2)/4) - ((sig_t[1,:]**2) * h[0] * theta) / (2 * (1 - theta)))
        h = np.delete(h, 0, axis=0)
        B = np.delete(B, 0, axis=0)
        sig_t = np.delete(sig_t, 0, axis=0)
        return Y_n, sig_t, B, h

In [ ]:
def Implicit(T, N, sig_0, M, alpha, Y_0, rho, theta):
    global h, sig_t, B
    dt = T/N # time step
    Y_n = np.ones(M)*Y_0 # array to hold values for Y at t for each monte carlo simulation
    sig_t = np.array([np.ones(M)*sig_0], dtype=float) # array to hold values for sigma at t for each monte carlo simulation
    A = np.array([alpha*rho, alpha*np.sqrt(1 - rho**2)], dtype=float) # Cholesky factor
    B_tminus1 = np.zeros((2,M)) 
    for i in range(N):
        h = np.array([0, dt], dtype=float) # array to hold time step
        B = np.array([np.sqrt(dt) * np.random.normal(0, 1, (2, M))], dtype=float) # Brownian motion
        B = np.insert(B, 0, B_tminus1, axis=0) # B(0) = 0
        B2 = A[0]*B[1, 0,:] + A[1]*B[1, 1,:] # Brownian motion for B1 in the SABR model
        sig_t = np.insert(sig_t, 1, sig_t[0,:]*np.exp((-alpha**2 / 2)*dt + B2), axis=0) # sigma(t)
        B1 = sig_t[1,:]*B[1, 0,:] # Brownian motion for B1 in the SABR model
        # If the square root is negative for Yn, half the step size and use a Brownian bridge
        if np.any((((Y_n + B1)**2)/4) < (((sig_t[1,:]**2) * dt * theta) / (2 * (1 - theta))))==True:
            sig_t = np.delete(sig_t, 1, axis=0) # delete sigma_n+1
            Y_n, sig_t, B, h = TimeStepAdaption(Y_n=Y_n, M=M, A=A, theta=theta)
            B_tminus1 = np.copy(B) # stores Brownian increment
  
        else:
            Y_n = ((Y_n + B1)/2) + np.sqrt((((Y_n + B1)**2)/4) - ((sig_t[1,:]**2) * dt * theta) / (2 * (1 - theta)))
            sig_t = np.delete(sig_t, 0, axis=0) # delete sigma_n
            B_tminus1 = np.copy(B[1,:,:])# store Brownian increment
        
    
    F_n = (Y_n * (1 - theta))**(1/(1 - theta)) # convet Yn to Fn
    return F_n